In [27]:
import os
import random
import warnings
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from sklearn import ensemble,neural_network,neighbors,svm,model_selection,inspection
warnings.simplefilter(action='ignore')

In [35]:
df = pd.read_csv('D:/ICARDA/TT2/RABPotato2.csv')
df

,longitude,latitude,plDate,hvDate,treat,treat2,yield,N_rate,P_rate,K_rate
0,29.327895,-1.583471,10/04/2021,02/05/2022,1,1,13.388072,51,22,0
1,29.327895,-1.583471,10/04/2021,02/05/2022,2,2,11.999783,0,0,0
2,29.327895,-1.583471,10/04/2021,02/05/2022,3,3,12.359695,90,40,60
3,29.327895,-1.583471,10/04/2021,02/05/2022,4,4,12.090858,51,0,42
4,29.327895,-1.583471,10/04/2021,02/05/2022,5,5,12.434738,51,22,42
...,...,...,...,...,...,...,...,...,...,...
650,30.058173,-1.650446,09/10/2021,01/03/2022,3,3,34.974074,90,40,60
651,30.058173,-1.650446,09/10/2021,01/03/2022,1,1,26.481260,51,22,0
652,30.058173,-1.650446,09/10/2021,01/03/2022,5,5,30.263950,51,22,42
653,30.058173,-1.650446,09/10/2021,01/03/2022,2,2,18.552497,0,0,0


In [36]:
xFeatures = ['longitude','latitude','treat2']

xFeaturesML = ['longitude','latitude','treat2']

xFeaturesHM = ['longitude','latitude','treat2']

yFeatur = 'yield'

In [37]:
inFile = 'D:/ICARDA/TT2'

In [38]:
outFolder = 'D:/ICARDA/TT2'

In [39]:
for i in [[xFeaturesML, 'ML'], [xFeaturesHM, 'HM']]:

  df = pd.read_csv('D:/ICARDA/TT2/RABPotato2.csv')
  xFeatures = i[0]
  group     = i[1]
  
  print(group)
  Xdf = df[xFeatures]
  ydf = df[[yFeatur]]

  for col in df.columns:
    if col not in ['plDate','hvDate','treat']:
      mean = np.mean(df[col].values)
      std  = np.std(df[col].values)
      if std != 0:
        df[col] = df[col].apply(lambda x:(x-mean)/std)
  
  testtreat = [2] # random.sample(list(df.treat.unique()), 1)
  print(testtreat)

  train = df[~df['treat'].isin(testtreat)]
  test  = df[df['treat'].isin(testtreat)]

  X_train = train[xFeatures]
  y_train = train[[yFeatur]]
  X_test  = test[xFeatures]
  y_test  = test[[yFeatur]]

  n = len(xFeatures)
  MLA = hyperparameters(n)
  for idMLA, mlaL_mlaA in enumerate(MLA.items()):
    mlaL, mlaA = mlaL_mlaA
    estimator = mlaA[0]
    param_grid = mlaA[1]
    GSCV = model_selection.GridSearchCV(estimator, param_grid, scoring='r2', cv=5, refit=True, verbose=False, n_jobs=-1)
    GSCV.fit(X_train, y_train)
    bestEstimator = GSCV.best_estimator_ 

    OriginalYield_test  = y_test.values.flatten()
    PredictedYield_test = bestEstimator.predict(X_test).flatten()
    OriginalYield_test  = (float(ydf.std()) * OriginalYield_test) + float(ydf.mean())
    PredictedYield_test = (float(ydf.std()) * PredictedYield_test) + float(ydf.mean())

    yieldFile   = os.path.join(outFolder, '{}_yield_{}_{}.xlsx'.format(mlaL, group, n))
    plot        = os.path.join(outFolder, '{}_plot_{}_{}.png'.format(mlaL, group, n))
    Stats       = os.path.join(outFolder, '{}_Stats_{}_{}.xlsx'.format(mlaL, group, n))
    Importances = os.path.join(outFolder, '{}_Importances_{}_{}.xlsx'.format(mlaL, group, n))

    dfYield = pd.DataFrame({'Original Yield':list(OriginalYield_test), 'Predicted Yield':list(PredictedYield_test)})
    dfYield.to_excel(yieldFile, index=False)

    plotOriginalPredicted(OriginalYield_test, PredictedYield_test, plot, label='Predicted yield of {}'.format(mlaL))
    df_stats = performanceStatistics(OriginalYield_test, PredictedYield_test)
    print(mlaL, df_stats)

    df_GSCV = pd.DataFrame(GSCV.cv_results_)
    df_GSCV = df_GSCV.sort_values(by='rank_test_score', ascending=True)
    df_GSCV = df_GSCV.head(1)
    for i in list(df_GSCV.columns):
      df_stats[i] = df_GSCV[i].values.tolist()[0]
    df_stats = pd.DataFrame.from_dict(df_stats, orient='index').T
    df_stats.to_excel(Stats, index=False)
    
    pi = inspection.permutation_importance(bestEstimator, X_train, y_train, n_jobs=-1, random_state=0).importances_mean
    pi = [((i / pi.sum()) * 100) for i in pi]
    dfImportances = pd.DataFrame(data=[pi], columns=xFeatures).round(2)
    dfImportances.to_excel(Importances, index=False)

ML
[2]
RFR {'R2': 0.13, 'RB': 92.42, 'MAE': 12.26, 'RMSD': 13.64, 'd': 0.43, 'n': 107}
ANN {'R2': 0.32, 'RB': 68.37, 'MAE': 9.11, 'RMSD': 9.62, 'd': 0.35, 'n': 107}
KNN {'R2': 0.22, 'RB': 72.66, 'MAE': 9.64, 'RMSD': 10.56, 'd': 0.45, 'n': 107}
SVR {'R2': 0.17, 'RB': 86.66, 'MAE': 11.5, 'RMSD': 12.45, 'd': 0.42, 'n': 107}
HM
[2]
RFR {'R2': 0.13, 'RB': 92.42, 'MAE': 12.26, 'RMSD': 13.64, 'd': 0.43, 'n': 107}
ANN {'R2': 0.32, 'RB': 68.37, 'MAE': 9.11, 'RMSD': 9.62, 'd': 0.35, 'n': 107}
KNN {'R2': 0.22, 'RB': 72.66, 'MAE': 9.64, 'RMSD': 10.56, 'd': 0.45, 'n': 107}
SVR {'R2': 0.17, 'RB': 86.66, 'MAE': 11.5, 'RMSD': 12.45, 'd': 0.42, 'n': 107}
